In [1]:
import pandas as pd
import numpy as np
import re
from PIL import Image
import webbrowser
import json
import pickle
import sys 
import joblib
import sys

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions as Reactions

from compound_cacher import CompoundCacher
from compound import Compound
from chemaxon import *
import chemaxon

In [2]:
def load_smiles():
    db = pd.read_csv('./../data/cache_compounds_20160818.csv',index_col='compound_id')
    db_smiles = db['smiles_pH7'].to_dict()
    return db_smiles

def load_molsig_rad1():
    molecular_signature_r1 = json.load(open('./../data/decompose_vector_ac.json'))
    return molecular_signature_r1

def load_molsig_rad2():
    molecular_signature_r2 = json.load(open('./../data/decompose_vector_ac_r2_py3_indent_modified_manual.json'))
    return molecular_signature_r2

def load_model():
    filename = './../model/M12_model_BR.pkl'
    loaded_model = joblib.load(open(filename, 'rb'))
    return loaded_model

In [3]:
db_smiles = load_smiles()
molsig_r1 = load_molsig_rad1()
molsig_r2 = load_molsig_rad2()
loaded_model = load_model()

In [4]:
def parse_reaction_formula_side(s):
    """
        Parses the side formula, e.g. '2 C00001 + C00002 + 3 C00003'
        Ignores stoichiometry.

        Returns:
            The set of CIDs.
    """
    if s.strip() == "null":
        return {}

    compound_bag = {}
    for member in re.split('\s+\+\s+', s):
        tokens = member.split(None, 1)
        if len(tokens) == 0:
            continue
        if len(tokens) == 1:
            amount = 1
            key = member
        else:
            amount = float(tokens[0])
            key = tokens[1]

        compound_bag[key] = compound_bag.get(key, 0) + amount

    return compound_bag

def parse_formula(formula, arrow='<=>', rid=None):
    """
        Parses a two-sided formula such as: 2 C00001 => C00002 + C00003

        Return:
            The set of substrates, products and the direction of the reaction
    """
    tokens = formula.split(arrow)
    if len(tokens) < 2:
        print(('Reaction does not contain the arrow sign (%s): %s'
                                 % (arrow, formula)))
    if len(tokens) > 2:
        print(('Reaction contains more than one arrow sign (%s): %s'
                                 % (arrow, formula)))

    left = tokens[0].strip()
    right = tokens[1].strip()

    sparse_reaction = {}
    for cid, count in parse_reaction_formula_side(left).items():
        sparse_reaction[cid] = sparse_reaction.get(cid, 0) - count

    for cid, count in parse_reaction_formula_side(right).items():
        sparse_reaction[cid] = sparse_reaction.get(cid, 0) + count  
    
    return sparse_reaction

In [5]:
rxn_string = "C00222 + C00010 + C00006 <=> C00024 + C00011 + C00005"

In [6]:
rxn_dic = parse_formula(rxn_string)

In [7]:
def get_ddG0(rxn_dict,pH,I,novel_mets):
    ccache = CompoundCacher()
    # ddG0 = get_transform_ddG0(rxn_dict, ccache, pH, I, T)
    T = 298.15
    ddG0_forward = 0
    for compound_id, coeff in rxn_dict.items():
        if novel_mets != None and compound_id in novel_mets:
            comp = novel_mets[compound_id]
        else:
            comp = ccache.get_compound(compound_id)
        ddG0_forward += coeff * comp.transform_pH7(pH, I, T)

    return ddG0_forward

In [8]:
get_ddG0(rxn_dic, 7.0,  0.1, {})

-3.6254822995515497

In [9]:
def get_rule(rxn_dict, molsig1, molsig2, novel_decomposed1, novel_decomposed2):
    if novel_decomposed1 != None:
        for cid in novel_decomposed1:
            molsig1[cid] = novel_decomposed1[cid]
    if novel_decomposed2 != None:
        for cid in novel_decomposed2:
            molsig2[cid] = novel_decomposed2[cid]

    molsigna_df1 = pd.DataFrame.from_dict(molsig1).fillna(0)
    all_mets1 = molsigna_df1.columns.tolist()
    all_mets1.append("C00080")
    all_mets1.append("C00282")

    molsigna_df2 = pd.DataFrame.from_dict(molsig2).fillna(0)
    all_mets2 = molsigna_df2.columns.tolist()
    all_mets2.append("C00080")
    all_mets2.append("C00282")

    moieties_r1 = open('./data/group_names_r1.txt')
    moieties_r2 = open('./data/group_names_r2_py3_modified_manual.txt')
    moie_r1 = moieties_r1.read().splitlines()
    moie_r2 = moieties_r2.read().splitlines()

    molsigna_df1 = molsigna_df1.reindex(moie_r1)
    molsigna_df2 = molsigna_df2.reindex(moie_r2)

    rule_df1 = pd.DataFrame(index=molsigna_df1.index)
    rule_df2 = pd.DataFrame(index=molsigna_df2.index)
    # for rid, value in reaction_dict.items():
    #     # skip the reactions with missing metabolites
    #     mets = value.keys()
    #     flag = False
    #     for met in mets:
    #         if met not in all_mets:
    #             flag = True
    #             break
    #     if flag: continue

    rule_df1['change'] = 0
    for met, stoic in rxn_dict.items():
        if met == "C00080" or met == "C00282":
            continue  # hydogen is zero
        rule_df1['change'] += molsigna_df1[met] * stoic

    rule_df2['change'] = 0
    for met, stoic in rxn_dict.items():
        if met == "C00080" or met == "C00282":
            continue  # hydogen is zero
        rule_df2['change'] += molsigna_df2[met] * stoic

    rule_vec1 = rule_df1.to_numpy().T
    rule_vec2 = rule_df2.to_numpy().T

    m1, n1 = rule_vec1.shape
    m2, n2 = rule_vec2.shape

    zeros1 = np.zeros((m1, 44))
    zeros2 = np.zeros((m2, 44))
    X1 = np.concatenate((rule_vec1, zeros1), 1)
    X2 = np.concatenate((rule_vec2, zeros2), 1)

    rule_comb = np.concatenate((X1, X2), 1)

    # rule_df_final = {}
    # rule_df_final['rad1'] = rule_df1
    # rule_df_final['rad2'] = rule_df2
    return rule_comb, rule_df1, rule_df2


In [14]:
rxn_dic

{'C00222': -1,
 'C00010': -1,
 'C00006': -1,
 'C00024': 1,
 'C00011': 1,
 'C00005': 1}

In [23]:
loaded_model.predict(X, return_std= True)

(array([-19.96775194]), array([6.66052556]))

In [ ]:
def get_ddG0(rxn_dict,pH,I,novel_mets):
    ccache = CompoundCacher()
    # ddG0 = get_transform_ddG0(rxn_dict, ccache, pH, I, T)
    T = 298.15
    ddG0_forward = 0
    for compound_id, coeff in rxn_dict.items():
        if novel_mets != None and compound_id in novel_mets:
            comp = novel_mets[compound_id]
        else:
            comp = ccache.get_compound(compound_id)
        ddG0_forward += coeff * comp.transform_pH7(pH, I, T)

    return ddG0_forward


def get_dG0(rxn_dict,rid,pH,I,loaded_model,molsig_r1, molsig_r2, novel_decomposed_r1, novel_decomposed_r2,novel_mets):
    rule_comb, rule_df1, rule_df2 = get_rule(rxn_dict,molsig_r1,molsig_r2, novel_decomposed_r1, novel_decomposed_r2)
    X  = rule_comb
    ymean, ystd = loaded_model.predict(X, return_std=True)
    result = {}
    return ymean[0] + get_ddG0(rxn_dict, pH, I, novel_mets),ystd[0], rule_df1, rule_df2

In [2]:
ccc=  CompoundCacher()

In [3]:
a = ccc.get_compound('C00001')

In [4]:
a.transform_pH7(7, 0.25 , 298)

81.4472134155519

In [5]:
inchi_k = "InChI=1S/C14H14O/c15-14-8-4-7-13(11-14)10-9-12-5-2-1-3-6-12/h1-8,11,15H,9-10H2" ;

In [6]:
c = Compound.from_inchi('Test', 'sajdf', inchi_k )

C:\Users\vuu10\AppData\Local\Continuum\anaconda3\envs\dGPredictor_py3\lib\openbabel\__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


In [18]:
c.smiles_ph7()

AttributeError: 'Compound' object has no attribute 'smiles_ph7'

In [7]:
from chemaxon import *
import chemaxon

In [8]:
pKas, major_ms_smiles = chemaxon.GetDissociationConstants(inchi_k)

In [9]:
major_ms_smiles

'OC1=CC=CC(CCC2=CC=CC=C2)=C1\r'

In [10]:
major_ms_smiles = Compound.smiles2smiles(major_ms_smiles)

In [11]:
MIN_PH = 0.0
MAX_PH = 14.0
pKas = sorted([pka for pka in pKas if pka > MIN_PH and pka < MAX_PH], reverse=True)

In [12]:
pKas

[10.1]

In [13]:
atom_bag, major_ms_charge = chemaxon.GetAtomBagAndCharge(major_ms_smiles)

In [21]:
formula, formal_charge = GetFormulaAndCharge(molstring)

atom_bag = {}

In [25]:
for mol_formula_times in formula.split('.'):
    for times, mol_formula in re.findall('^(\d+)?(\w+)', mol_formula_times):
        if not times:
            times = 1
        else:
            times = int(times)
        for atom, count in re.findall("([A-Z][a-z]*)([0-9]*)", mol_formula):
            if count == '':
                count = 1
            else:
                count = int(count)
            atom_bag[atom] = atom_bag.get(atom, 0) + count * times

In [26]:
atom_bag

{'C': 14, 'H': 14, 'O': 1}

In [52]:
from rdkit.Chem import rdchem
for (elem, c) in atom_bag.items():
    ll = rdchem.GetPeriodicTable()
    atomic_num = ll.GetAtomicNumber(elem)
    print(atomic_num)

6
1
8


In [55]:

n_protons = sum([c * ll.GetAtomicNumber(str(elem))
                 for (elem, c) in atom_bag.items()])

In [57]:
atom_bag['e-'] = n_protons - formal_charge

In [58]:
atom_bag

{'C': 14, 'H': 14, 'O': 1, 'e-': 106}

In [60]:

formal_charge



0

In [ ]:
all_pKas, smiles_list = GetDissociationConstants_val(inchi_k)

In [13]:
MID_PH = 7.0
N_PKAS = 20

n_acidic = N_PKAS
n_basic = N_PKAS
pH = MID_PH

In [14]:
args = []
if n_acidic + n_basic > 0:
    args += ['pka', '-a', str(n_acidic), '-b', str(n_basic),
             'majorms', '-M', 'true', '--pH', str(pH)]


In [15]:
args

['pka', '-a', '20', '-b', '20', 'majorms', '-M', 'true', '--pH', '7.0']

In [16]:
logging.debug("INPUT: echo %s | %s" % (inchi_k, ' '.join([CXCALC_BIN] + args)))

In [17]:
molstring= inchi_k

In [18]:
p1 = Popen(["echo", molstring], stdout=PIPE, shell=use_shell_for_echo)

In [19]:
p2 = Popen([CXCALC_BIN] + args, stdin=p1.stdout,
                   executable=CXCALC_BIN, stdout=PIPE, shell=False)

In [20]:
res = p2.communicate()[0]

In [21]:
if p2.returncode != 0:
    raise ChemAxonError(str(args))
logging.debug("OUTPUT: %s" % res)

In [22]:
output = res

In [23]:
output

b'id\tapKa1\tapKa2\tapKa3\tapKa4\tapKa5\tapKa6\tapKa7\tapKa8\tapKa9\tapKa10\tapKa11\tapKa12\tapKa13\tapKa14\tapKa15\tapKa16\tapKa17\tapKa18\tapKa19\tapKa20\tbpKa1\tbpKa2\tbpKa3\tbpKa4\tbpKa5\tbpKa6\tbpKa7\tbpKa8\tbpKa9\tbpKa10\tbpKa11\tbpKa12\tbpKa13\tbpKa14\tbpKa15\tbpKa16\tbpKa17\tbpKa18\tbpKa19\tbpKa20\tatoms\tmajor-ms\r\n1\t10.10\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t-5.48\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t15,15\tOC1=CC=CC(CCC2=CC=CC=C2)=C1\r\n'

In [24]:
atom2pKa, smiles_list = ParsePkaOutput(output, n_acidic, n_basic)

In [26]:
smiles_list

['OC1=CC=CC(CCC2=CC=CC=C2)=C1\r']

In [27]:
all_pKas = []
for pKa_list in list(atom2pKa.values()):
    all_pKas += [pKa for pKa, _ in pKa_list]

In [28]:
all_pKas

[10.1, -5.48]